----
# Exponential Smoothing
-----

Exponential Smoothing is a forecasting method that applies greater weight to more recent observations while exponentially decreasing the weight of older data points. In this notebook, I will explore and compare the three exponential smoothing methods using the **holtwinters** package in Python:

**1. Simple Exponential Smoothing:**

This method is used for time series data without trend or seasonality. 

It averages past data, giving more weight to recent observations and less to older ones. The smoothing parameter (alpha) adjusts how much emphasis is placed on recent data and is used to set the level (a baseline for predictions, similar to the Naive Forecasting method). 

**2. Holt’s Linear Trend Model (Double Exponential Smoothing):**

This method builds on simple exponential smoothing by adding a trend component (another paramter). This method provides predictions that take into account the level and direction of trend.

**3. Holt-Winters Seasonal Model (Triple Exponential Smoothing):**

This model builds on the Holt's Linear Model by adding a seasonality component. This method provides predictions that take into account the level and direction of trend as well as the observed seasonality in data.

## Set Up
---

In [17]:
import numpy as np
import pandas as pd

# plotting
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns

# stats
from statsmodels.api import tsa # time series analysis
import statsmodels.api as sm
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, Holt, ExponentialSmoothing

# evaluate
from sklearn.metrics import mean_squared_error, mean_absolute_error

## Utility Functions
----

In [116]:
def plt_forecast(predictions, fc_method):
    """
    Description:
    Plots the training data, validation data (actual), and baseline predictions on a single graph.

    Parameters:
    - predictions : A Series containing the predicted values with date indices.
    - fc_method: A string describing the forecasting method used.

    Output:
    The function creates a plot using Plotly to visualise:
        - Training data
        - Validation data 
        - Baseline forecast predictions

    """
    
    # Plot to visualise the training data, test data and baseline prediction
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=train_df.index, y=train_df['Adj Close'], mode='lines', name="Train"))
    fig.add_trace(go.Scatter(x=val_df.index, y=val_df['Adj Close'], mode='lines', name="Validation"))
    fig.add_trace(go.Scatter(x=predictions.index, y=predictions, mode='lines', name="Forecast"))

    fig.update_layout(
        yaxis_title='Adjusted Close', 
        xaxis_title='Date',
        title= f'{fc_method}'
    )
    fig.show()

In [117]:
def fcast_evaluation(predicted, actual):
    """
    Description:
    To evaluate forecasting performance using multiple metrics.

    Parameters:
    predicted: Forecasted values.
    actual: Actual observed values.

    Output:
    A dictionary containing the evaluation metrics:
        - 'MSE': Mean Squared Error
        - 'MAE': Mean Absolute Error
        - 'RMSE': Root Mean Squared Error
        - 'MAPE': Mean Absolute Percentage Error
    """

    err= actual - predicted

    # Calculating MSE
    mse = mean_squared_error(actual, predicted)

    # Calculating MAE
    mae = mean_absolute_error(actual, predicted)

    # Calculating RMSE
    rmse = np.sqrt(mse)

    # Calculating MAPE
    abs_percent_err = np.abs(err/actual)
    mape = abs_percent_err.mean() * 100

    return {'MSE': mse,
            'MAE': mae,
            'RMSE': rmse,
            'MAPE': mape
            }

## Data Loading
----

In [118]:
raw_data = pd.read_csv('../../data/daily_data_clean.csv', index_col=0)

In [119]:
# Filter the data range to the past year for a closer inspection of the prediction
raw_data_filtered = raw_data.loc[(raw_data.index >= '2023-07-29'), ['Adj Close']]

In [120]:
# Splitting of the data into train/val datasets
train_df = raw_data_filtered.loc[raw_data_filtered.index <= "2024-04-29"]
val_df = raw_data_filtered.loc[raw_data_filtered.index > "2024-04-29"]

## Simple Exponential Smoothing
---

In [121]:
# Ignore warning as D is correct inference
simp_exp_smooth = SimpleExpSmoothing(train_df['Adj Close'])
simp_model = simp_exp_smooth.fit(optimized = True)
forecasts = simp_model.forecast(val_df.shape[0])

/Users/simybasra/anaconda3/envs/timeseries/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency D will be used.



In [122]:
plt_forecast(forecasts,'Simple Exponential Smoothing')

In [134]:
simp_model.summary()

Dep. Variable:,Adj Close,No. Observations:,276
Model:,SimpleExpSmoothing,SSE,3393.880
Optimized:,True,AIC,696.575
Trend:,None,BIC,703.815
Seasonal:,None,AICC,696.722
Seasonal Periods:,None,Date:,"Fri, 30 Aug 2024"
Box-Cox:,False,Time:,15:41:10
Box-Cox Coeff.:,None,,
,coeff,code,optimized
smoothing_level,0.8753711,alpha,True
initial_level,334.93326,l.0,False


-----
**Comment:**

The smoothing_level parameter is very close to 1 suggesting that the model is heavily relying on the most recent data points for forecasting. This is similar to the naive baseline model where future predictions are based on the most recent observation.

In [124]:
fcast_evaluation(forecasts.values ,val_df['Adj Close'].values)

{'MSE': 1377.4494179959815,
 'MAE': 32.8262220309689,
 'RMSE': 37.11400568513161,
 'MAPE': 7.411262366988847}

----
**Comment:**

Simple Exponential Smoothing did not provide a marked improvement in forecasting accuracy. This suggests that more complex or alternative methods may be needed to enhance performance.

## Holt's Linear Trend Model
----

In [125]:
holt_lin = Holt(train_df['Adj Close'], damped_trend=True)
holt_lin_model = holt_lin.fit(optimized = True)
forecast_2 = holt_lin_model.forecast(val_df.shape[0])

/Users/simybasra/anaconda3/envs/timeseries/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency D will be used.



In [133]:
plt_forecast(forecast_2,'Holt\'s Linear Trend Model')

In [127]:
holt_lin_model.summary()

Dep. Variable:,Adj Close,No. Observations:,276
Model:,Holt,SSE,3364.118
Optimized:,True,AIC,700.144
Trend:,Additive,BIC,718.246
Seasonal:,None,AICC,700.562
Seasonal Periods:,None,Date:,"Fri, 30 Aug 2024"
Box-Cox:,False,Time:,15:40:57
Box-Cox Coeff.:,None,,
,coeff,code,optimized
smoothing_level,0.8345351,alpha,True
smoothing_trend,0.0350526,beta,True


-----
**Comment:**

Smoothing level parameter is still at a high value, indicating strong reliance on recent data for estimating the level.

Smoothing trend parameter is a rather low value, suggesting the model assumes the trend is relatively stable with a slight increase over time. This may result in the model to lag in capturing changes in the trend, this can be seen if we compare the forecasted values to the actual values.

In [129]:
fcast_evaluation(forecast_2.values ,val_df['Adj Close'].values)

{'MSE': 1525.3241347710375,
 'MAE': 34.81695731016647,
 'RMSE': 39.05539827950852,
 'MAPE': 7.866536282033189}

-----
**Comment:**

Overall, the Holt Linear Trend method performed worse than the Drift Model. Despite incorporating trend components into exponential smoothing, it did not provide an improvement in the forecasting accuracy. 

Overall, the Holt Linear Trend method performed worse than the Drift Model. Despite its ability to incorporate trend components, the accuracy of its predictions did not increase. This suggests that the Holt Linear Trend method may not be capturing the underlying patterns effectively for stock data.

## Holt Winter's Method
----

In [130]:
exp_smooth = ExponentialSmoothing(train_df['Adj Close'],
                                  trend = 'add',
                                  seasonal= 'add',
                                  seasonal_periods= 7)
model = exp_smooth.fit(optimized=True)
forecasting = model.forecast(val_df.shape[0])

/Users/simybasra/anaconda3/envs/timeseries/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency D will be used.



In [131]:
plt_forecast(forecasting, 'Holt Winter\'s Method')

In [132]:
model.summary()

Dep. Variable:,Adj Close,No. Observations:,276
Model:,ExponentialSmoothing,SSE,3359.093
Optimized:,True,AIC,711.731
Trend:,Additive,BIC,751.555
Seasonal:,Additive,AICC,713.120
Seasonal Periods:,7,Date:,"Fri, 30 Aug 2024"
Box-Cox:,False,Time:,15:41:02
Box-Cox Coeff.:,None,,
,coeff,code,optimized
smoothing_level,0.8687112,alpha,True
smoothing_trend,2.4667e-07,beta,True


----
**Comment:**

Smoothing level paramter is high, again showing a strong reliance on the recent data for forecasting.

Smoothing trend paramter is even lower, the model assumes a stable trend.

Smoothing seasonal component is extremely low, suggesting the model may not effectively capture the seasonal weekly patterns discovered in eda.

In [115]:
fcast_evaluation(forecasting.values ,val_df['Adj Close'].values)

{'MSE': 651.1175023931354,
 'MAE': 21.719282150022977,
 'RMSE': 25.51700418139119,
 'MAPE': 4.894284824685268}

-----
**Comment:**

MSE = 651.12: Suggests a considerable error level in predictions but still needs improvement.

MAE = 21.72: Shows that, on average, the forecast deviates by about 22 units from the actual values, similar level to the Drift Baseline Model.

RMSE = 25.52: Again, fairly similar to the Drift Model, not much improvement made.

MAPE = 4.89%: Slightly lower result than Drift Model, but still need the MAPE of forecasted to be a lot lower.

Despite the added complexity of exponential smoothing, the performance improvements over baseline methods are marginal. The models do not provide significant benefits compared to simpler approaches.

## Conclusion
-----

The exponential smoothing methods explored in this notebook did not perform well with the stock data. Advanced forecasting models such as ARIMA may be needed in order to better capture the complexities of stock data.